In [769]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd

In [770]:
#reading rheast and extracting districts
rheast= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
pre=pd.read_csv('data/preharvest/preharvest_cleaned.csv')
post=pd.read_csv('data/postharvest/postharvest_cleaned.csv')
# post['province']=post['province'].replace(['northwest'], 'northwest')

In [771]:
# # Read shapefiles and rename columns
# # country_shapefile = gpd.read_file("data/shp/gadm36_ZMB_0.shp")
# # district_shp = gpd.read_file("data/shp/gadm36_ZMB_2.shp")


# # Rename kenya_shp 
# district_shp=district_shp.rename(columns={'NAME1':'cname'})
# # district_shp['cname']=district_shp['NAME_2']
# # len(district_shp.cname.unique())
# district_shp['cname']=district_shp['cname'].str.lower()
# district_shp['cname']=district_shp['cname'].str.replace('-', '', regex=True)
# district_shp['cname']=district_shp['cname'].str.strip()
# len(district_shp.cname.unique())

In [772]:
# # remove all white spaces, turn to lower case and
rheast.cname=rheast.cname.str.lower()
rheast.cname = rheast.cname.str.strip()
rheast.cname=rheast.cname.str.replace('-', '', regex=True)
rheast.cname=rheast.cname.str.replace(' ', '', regex=True)

# pre.cname=pre.cname.str.lower()
# pre.cname = pre.cname.str.strip()
# pre.cname=pre.cname.str.replace('-', '', regex=True)
# pre.cname=pre.cname.str.replace(' ', '', regex=True)
# pre
# post

In [773]:
rheast['district']=rheast['cname']

In [774]:
#Extracting Districts
pre_list_districts= pre['district'].to_list()
pre_set_districts=set(pre_list_districts)
pre_districts=list(pre_set_districts)
rheast_list_districts=rheast['district'].to_list()
rheast_set_districts=set(rheast_list_districts)
rheas_districts=list(rheast_set_districts)

In [775]:
#Finding common districts in rheas and preharvest rheast
districts=[]
for i in pre_districts:
    if i in rheas_districts:
        districts.append(i)
    else:
        print(i)

missing_districts=[]
for j in rheas_districts:
    if j not in  pre_districts:
        missing_districts.append(j)
len(missing_districts)
rheast.cname.unique()

array(['kalabo', 'kalomo', 'kasama', 'kalumbila', 'kafue', 'kaputa',
       'kalulushi', 'kapirimposhi', 'kanchibiya', 'kasempa', 'kaoma',
       'kabwe', 'kitwe', 'luangwa', 'kawambwa', 'katete', 'lavushimanda',
       'luampa', 'kasenengwa', 'limulunga', 'livingstone', 'kazungula',
       'mansa', 'luwingu', 'mafinga', 'manyinga', 'milengi', 'mambwe',
       'mitete', 'mazabuka', 'mbala', 'mkushi', 'shangombo', 'chavuma',
       'chembe', 'sesheke', 'chadiza', 'chama', 'chasefu', 'chibombo',
       "shiwang'andu", 'shibuyunji', 'chinsali', 'chiengi', 'chilubi',
       'chilanga', 'chingola', 'chipata', 'chifunabuli', 'chililabombwe',
       'chipangali', 'chikankanta', 'monze', 'mpulungu', 'mufulira',
       'mungwi', 'mufumbwe', 'mongu', 'mpongwe', 'mulobezi', 'mumbwa',
       'mpika', 'mporokoso', 'kabompo', 'chisamba', 'isoka', 'gwembe',
       'chipili', 'chitambo', 'ikelenge', 'itezhitezhi', 'choma',
       'chongwe', 'chirundu', 'zimba', 'siavonga', 'sinda', 'vubwi',
       'so

In [776]:
#changing date columns
rheast['planting']=pd.to_datetime(rheast['planting'], format='ISO8601')
rheast['harvest'] = pd.to_datetime(rheast['harvest'], format='ISO8601')
rheast['year'] = rheast['planting'].dt.year
rheast['yield']=rheast['gwad']/1000


In [777]:
#Derive seasons from the data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
rheast['season']= rheast['planting'].dt.month.map(month_to_season)

# Extract data only for the planting season
rheast_model= rheast.loc[rheast['season']==1]
rheast_data=rheast_model.groupby(['district','year', 'season'], as_index=False).agg({'yield': 'mean'})

pre1=pre.groupby(['district', 'year'], as_index=False).agg({'yield': 'mean'})
# rheast_data[['district', 'year', 'yield']].to_csv('rheas_dssat_model.csv')
rheast_data

,district,year,season,yield
0,chadiza,1999,1,4.25366
1,chadiza,2000,1,4.17192
2,chadiza,2001,1,4.23727
3,chadiza,2002,1,4.23210
4,chadiza,2003,1,3.99673
...,...,...,...,...
2779,zimba,2018,1,3.80788
2780,zimba,2019,1,3.74285
2781,zimba,2020,1,3.99101
2782,zimba,2021,1,4.16769


In [778]:
# only from 2011 to 2022
rheast_data=rheast_data.loc[rheast_data['year']>=2011]
rheast_district=pd.DataFrame()
distric_metrics1=[]
for i in districts:
        pre2=pre1.loc[pre1['district']==i].reset_index(drop=True)
        rheast_data3=rheast_data.loc[rheast_data['district']==i].reset_index(drop=True)
        #  pearson's correlation
        corr=rheast_data3['yield'].corr(pre2['yield'])
        # # #root mean square error
        rmse = np.sqrt(((rheast_data3['yield']-pre2['yield'])**2).mean())
        # # #unbiased RMSE
        ubrmse = np.sqrt((((rheast_data3['yield']-rheast_data3['yield'].mean())
                    -(pre2['yield']-pre2['yield'].mean()))**2).mean())
        # #bias
        bias = (rheast_data3['yield']-pre2['yield']).mean()
        distric_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
        rheast_df=pd.concat([rheast_data3,pre2], axis=0)
        # convert into dataframe and rename column
        district_statistics1=pd.DataFrame(distric_metrics1)
        district_statistics2=district_statistics1.rename({0:'district',1: 'correlation', 2: 
                                                    'rmse', 3:'ubrmse', 4:'bias'}, axis=1)

# district_statistics1.to_csv('data/preharvest_rheas__district_metrics.csv')
# rheast_df
rheast_data=rheast_data.groupby(['district'],as_index=False).agg({'yield':'min'})
# pre2=pre2.groupby(['district'],as_index=False).agg({'yield':'min'})
district_statistics2


,district,correlation,rmse,ubrmse,bias
0,gwembe,0.552920,1.904615,0.722073,1.762433
1,ikelenge,-0.371525,1.162132,0.424636,1.081775
2,siavonga,0.530908,2.212277,0.388354,2.177923
3,mporokoso,0.260625,0.953312,0.291100,0.907780
4,mungwi,0.197602,0.642267,0.241984,0.594938
...,...,...,...,...,...
69,kabwe,0.438931,1.013266,0.672973,0.757506
70,lundazi,0.181003,1.436995,0.376791,1.386716
71,chipata,0.344599,1.645490,0.219095,1.630839
72,kabompo,-0.211252,1.199537,0.392070,1.133654


In [779]:
# Province Level
# rename columns
pre['district_b']=pre['district']
rheast_data['district_b']=rheast_data['district']
post['province_b']=post['province']


#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('data/shp/zmb_district_cleaned.shp')
zambia_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('data/shp/zmb_province_cleaned.shp')

# merge the yields and shapefiles by provinces
pre_shp= pd.merge(zambia_district_shp, pre, on="district_b")
post_shp= pd.merge(zambia_province_shp, post, on="province_b")
rheast_shp= pd.merge(zambia_district_shp, rheast_data, on="district_b")

# aggregate preharvest to provincial level 
pre_provincial=pre_shp.groupby(['province_b','province', 'year'], as_index=False).agg({'harvested_ha':'sum', 'production_mt': 
                                                             'sum','planted_ha':'sum' })
pre_provincial['yield']=pre_provincial['production_mt']/pre_provincial['harvested_ha']
pre_provincial['yield_planted']=pre_provincial['production_mt']/pre_provincial['planted_ha']
rheast_province=rheast_shp.groupby(['province_b', 'province', 'year'], as_index=False).agg({'yield':'mean'})
post_province=post_shp.groupby(['province_b', 'year'], as_index=False).agg({'yield':'mean'})

KeyError: 'year'

In [ ]:
provinces=['central', 'copperbelt', 'eastern', 'luapula', 'lusaka',
       'muchinga', 'northern', 'northwestern', 'southern', 'western']

province_metrics1=[]
for i in provinces:
    pre_province1=pre_provincial.loc[pre_provincial['province_b']==i].reset_index(drop=True)
    rheast_province1=rheast_province.loc[rheast_province['province_b']==i].reset_index(drop=True)  
    #  pearson's correlation
    corr=rheast_province1['yield'].corr(pre_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province1['yield']-pre_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province1['yield']-rheast_province1['yield'].mean())
                       -(pre_province1['yield']-pre_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province1['yield']-pre_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics1=pd.DataFrame(province_metrics1)
province_statistics1=province_statistics1.rename({0:'province',1: 'correlation',2: 
                                                  'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics1.to_csv('preharvest_dssat_province_level_metrics.csv')
# province_statistics1.to_csv('data/preharvest_rheas__province_metrics.csv')

In [ ]:
# Postharvest and rheas common years 
post_list_years= post['year'].to_list()
years_set=set(post_list_years)
post_harvest_years=list(years_set)
post_harvest_years

# df1=rheast_province.loc[rheast_province['year']==2016]
rheast_province2=pd.DataFrame(columns =[rheast_province.columns])
for i in post_harvest_years:
    df=rheast_province.loc[rheast_province['year']==i].reset_index(drop=True)
    rheast_province2=pd.concat([df])


In [ ]:
# Postharvest and rheas metrics
# only data from 2011 to 2014

rheast_province=rheast_province.loc[(rheast_province['year']>=2011)|(rheast_province['year']<=2014)]

province_metrics2=[]
for i in provinces:
    post_province1=post_province.loc[post_province['province_b']==i].reset_index(drop=True)
    rheast_province2=rheast_province.loc[rheast_province['province_b']==i].reset_index(drop=True)

    #  pearson's correlation
    corr=rheast_province2['yield'].corr(post_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province2['yield']-post_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province2['yield']-rheast_province2['yield'].mean())
                       -(post_province1['yield']-post_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province2['yield']-post_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics2.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics2=pd.DataFrame(province_metrics2)
province_statistics2=province_statistics2.rename({0:'province',1: 
                                                  'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics.to_csv('preharvest_dssat_province_level_metrics.csv')
# province_statistics2.to_csv('data/postharvest_rheas_province_level_metrics.csv')

province_statistics2

,province,correlation,rmse,ubrmse,bias
0,central,0.744452,1.968286,0.206634,1.958162
1,copperbelt,0.850875,1.837365,0.129059,1.833665
2,eastern,-0.163787,2.279473,0.108891,2.277634
3,luapula,0.638092,1.485551,0.075239,1.484536
4,lusaka,0.404531,1.919803,0.103091,1.917688
5,muchinga,0.588150,1.432830,0.143487,1.426751
6,northern,0.973120,1.607271,0.155930,1.600475
7,northwestern,0.896474,1.831956,0.240573,1.816671
8,southern,0.189236,2.721992,0.159896,2.717731
9,western,-0.345091,3.310596,0.246453,3.301880


In [ ]:
# bias correction

# rheast_province['yield_corrected']
rheast_province=rheast_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province.rename({'province_b':'province_p','yield':'postharvest'},axis=1, inplace=True)
rheast_province['yield_corrected']=rheast_province['yield']-province_statistics2['bias']
rheas_postharvest_provincial=pd.concat([rheast_province,post_province,province_statistics2], axis=1)
# rheas_postharvest_provincial.rename({'yield':'rheas', 'yield':'postharvest'},axis=1, inplace=True)
rheas_postharvest_provincial

,province_b,yield,yield_corrected,province_p,postharvest,province,correlation,rmse,ubrmse,bias
0,central,3.982243,2.024082,central,2.078341,central,0.744452,1.968286,0.206634,1.958162
1,copperbelt,3.988369,2.154704,copperbelt,2.210160,copperbelt,0.850875,1.837365,0.129059,1.833665
2,eastern,3.986874,1.709240,eastern,1.768208,eastern,-0.163787,2.279473,0.108891,2.277634
3,luapula,3.987465,2.502929,luapula,2.554374,luapula,0.638092,1.485551,0.075239,1.484536
4,lusaka,3.983016,2.065328,lusaka,2.115439,lusaka,0.404531,1.919803,0.103091,1.917688
5,muchinga,3.992105,2.565354,muchinga,2.621986,muchinga,0.588150,1.432830,0.143487,1.426751
6,northern,3.991130,2.390655,northern,2.440774,northern,0.973120,1.607271,0.155930,1.600475
7,northwestern,3.987305,2.170634,northwestern,2.216512,northwestern,0.896474,1.831956,0.240573,1.816671
8,southern,3.983119,1.265388,southern,1.314307,southern,0.189236,2.721992,0.159896,2.717731
9,western,3.982349,0.680469,western,0.736167,western,-0.345091,3.310596,0.246453,3.301880
